In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/opt/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
diamonds = pd.read_csv('../data/diamonds_train.csv')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95


Diamond_Volume = mass/density. with this in mind, we could replace a new "volume" column for X,Y,Z

In [6]:
diamonds['volume']=diamonds['carat'].apply(lambda x: x/3.51)

In [7]:
diamonds_vol = diamonds[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'volume']]
diamonds_vol.head()

,carat,cut,color,clarity,depth,table,price,volume
0,1.21,Premium,J,VS2,62.4,58.0,4268,0.344729
1,0.32,Very Good,H,VS2,63.0,57.0,505,0.091168
2,0.71,Fair,G,VS1,65.5,55.0,2686,0.202279
3,0.41,Good,D,SI1,63.8,56.0,738,0.116809
4,1.02,Ideal,G,SI1,60.5,59.0,4882,0.290598


In [8]:
diamonds_vol.to_csv('../data/diamonds_vol.csv')

data loading

In [9]:
diamonds_vol = pd.read_csv('../data/diamonds_vol.csv')
diamonds_predict = pd.read_csv('../data/diamonds_predict.csv')

In [33]:
diamonds_predict['volume']=diamonds_predict['carat'].apply(lambda x: x/3.51)

In [35]:
diamonds_predict_vol = diamonds_predict[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'volume']]
diamonds_predict_vol.head()

,carat,cut,color,clarity,depth,table,volume
0,0.79,Very Good,F,SI1,62.7,60.0,0.225071
1,1.20,Ideal,J,VS1,61.0,57.0,0.341880
2,1.57,Premium,H,SI1,62.2,61.0,0.447293
3,0.90,Very Good,F,SI1,63.8,54.0,0.256410
4,0.50,Very Good,F,VS1,62.9,58.0,0.142450


In [36]:
diamonds_predict_vol.to_csv('../data/diamonds_predict_vol.csv')

ml_processing

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [11]:
target = 'price'
cat_features = ['cut', 'color', 'clarity']
num_features = ['carat', 'depth', 'table', 'volume']
features = num_features + cat_features

Strategy=median

In [12]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
                ('scaler', StandardScaler())])

In [13]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [14]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, num_features),
                                ('cat', categorical_transformer, cat_features)])

In [15]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'table', 'volume']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['cut', 'color', 'clarity'])])

In [16]:
pd.DataFrame(data=preprocessor.fit_transform(diamonds_vol).todense()).head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.867006,0.452019,0.247981,0.867006,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.004557,0.871099,-0.199745,-1.004557,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.184434,2.617265,-1.095198,-0.184434,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.815298,1.429872,-0.647472,-0.815298,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.467458,-0.875068,0.695707,0.467458,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
diamonds_train, diamonds_test = train_test_split(diamonds_vol)

In [19]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 9)
(10114, 9)


In [20]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', RandomForestRegressor())])

In [21]:
model.fit(diamonds_train[features], diamonds_train[target]);

model performance..

In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
y_test = model.predict(diamonds_test[features])
y_train = model.predict(diamonds_train[features])

In [24]:
print(f"test error: {mean_squared_error(y_pred=y_test, y_true=diamonds_test[target], squared=False)}")
print(f"train error: {mean_squared_error(y_pred=y_train, y_true=diamonds_train[target], squared=False)}")

test error: 581.4515816382846
train error: 212.5458405997895


cross validation

In [25]:
from sklearn.model_selection import cross_val_score

In [26]:
scores = cross_val_score(model, 
                         diamonds_vol[features], 
                         diamonds_vol[target], 
                         scoring='neg_root_mean_squared_error', 
                         cv=5, n_jobs=-1)

In [27]:
import numpy as np
np.mean(-scores)

557.3081114269274

grid search

In [28]:
from sklearn.model_selection import RandomizedSearchCV

In [29]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8, 16],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(diamonds_vol[features], diamonds_vol[target])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 23.6min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 31.4min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'depth',
                                                                                'table',
                                                                                'volume']),
       

submission

In [41]:
grid_search.best_params_

{'regressor__n_estimators': 512,
 'regressor__max_depth': 16,
 'preprocessor__num__imputer__strategy': 'mean'}

In [42]:
grid_search.best_score_

-551.1821035024572

submission

In [83]:
y_pred = grid_search.predict(diamonds_predict_vol[features])

In [84]:
diamonds_predict_vol['features'] = y_pred

/opt/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
submission_vol_1509 = diamonds_predict_vol[['features']]

In [86]:
submission_vol_1509 = pd.DataFrame({'price': y_pred})

In [87]:
submission_vol_1509.index.name='id'

In [88]:
submission_vol_1509

,price
id,
0,3002.043007
1,5359.811380
2,10651.644608
3,4147.804510
4,1633.007971
...,...
13480,1702.379543
13481,2366.727888
13482,3112.147207


In [89]:
submission_vol_1509.head()

,price
id,
0,3002.043007
1,5359.811380
2,10651.644608
3,4147.804510
4,1633.007971


In [90]:
submission_vol_1509.describe()

,price
count,13485.000000
mean,3953.524148
std,3954.377214
min,366.524115
25%,940.229177
50%,2454.017206
75%,5306.195735
max,18171.390318


In [91]:
submission_vol_1509.price.clip(0, 20000, inplace=True)

In [92]:
submission_vol_1509.to_csv('submission_vol_1509.csv')